## Dog Breed classifier using Transfer Learning and tf.data


In [0]:
#"""
# Google Collab specific stuff....
from google.colab import drive
drive.mount('/content/drive')

import os
!ls "/content/drive/My Drive"

USING_COLLAB = True
%tensorflow_version 2.x
#"""

In [0]:
# Setup sys.path to find MachineLearning lib directory

try: USING_COLLAB
except NameError: USING_COLLAB = False

%load_ext autoreload
%autoreload 2

import sys
if "MachineLearning" in sys.path[0]:
    pass
else:
    print(sys.path)
    if USING_COLLAB:
        sys.path.insert(0, '/content/drive/My Drive/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    else:
        sys.path.insert(0, '/Users/john/Documents/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
    print(sys.path)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv, gc
import numpy as np 

import IPython.display as display
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange, tqdm
import pandas as pd

import tensorflow as tf
print(tf.__version__)

import tensorflow_datasets as tfds

AUTOTUNE = tf.data.experimental.AUTOTUNE
print("AUTOTUNE: ", AUTOTUNE)

from TrainingUtils import *

#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

## Examine and understand data


In [0]:
# GLOBALS/CONFIG ITEMS

# Set root directory path to data
if USING_COLLAB:
    ROOT_PATH = "/content/drive/My Drive/ImageData/DogBreeds"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
else:
    ROOT_PATH = "/Users/john/Documents/ImageData/DogBreeds"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
        
# Establish global dictionary
parms = GlobalParms(MODEL_NAME="model-DogBreeds-tf-data-V02.h5",
                    ROOT_PATH=ROOT_PATH,
                    TRAIN_DIR="train", 
                    SMALL_RUN=False,
                    NUM_CLASSES=120,
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=32,
                    EPOCS=10,  # change to larger to improve results
                    IMAGE_EXT=".jpg",
                    FINAL_ACTIVATION='softmax',
                    LOSS='categorical_crossentropy',
                    METRICS=['accuracy'])

parms.print_contents()

In [0]:
# Simple helper method to display batches of images with labels....        
def show_batch(image_batch, label_batch, number_to_show=25, r=5, c=5, print_shape=False):
    show_number = min(number_to_show, parms.BATCH_SIZE)

    if show_number < 8: #if small number, then change row, col and figure size
        if parms.IMAGE_COLS > 64 or parms.IMAGE_ROWS > 64:
            plt.figure(figsize=(25,25)) 
        else:
            plt.figure(figsize=(10,10))  
        r = 4
        c = 2 
    else:
        plt.figure(figsize=(10,10))  

    if show_number == 1:
        image_batch = np.expand_dims(image_batch, axis=0)
        label_batch = np.expand_dims(label_batch, axis=0)

    for n in range(show_number):
        if print_shape:
            print("Image shape: {}  Max: {}  Min: {}".format(image_batch[n].shape, np.max(image_batch[n]), np.min(image_batch[n])))
        ax = plt.subplot(r,c,n+1)
        cmap="gray"
        if len(image_batch[n].shape) == 3:
            if image_batch[n].shape[2] == 3:
                cmap="viridis"
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n]), cmap=plt.get_cmap(cmap))
        plt.title(parms.CLASS_NAMES[np.argmax(label_batch[n])])
        plt.axis('off')

In [0]:
# Download dataset to local VM
datasets, info = tfds.load(name='stanford_dogs', with_info=True, as_supervised=True)


In [0]:
# Set Class names...
parms.set_class_names(['Chihuahua', 'Japanese_spaniel', 'Maltese_dog', 'Pekinese', 'Shih-Tzu', 'Blenheim_spaniel', 'papillon', 'toy_terrier', 'Rhodesian_ridgeback', 'Afghan_hound', 'basset', 'beagle', 'bloodhound', 'bluetick', 'black-tan_coonhound', 'Walker_hound', 'English_foxhound', 'redbone', 'borzoi', 'Irish_wolfhound', 'Italian_greyhound', 'whippet', 'Ibizan_hound', 'Norwegian_elkhound', 'otterhound', 'Saluki', 'Scottish_deerhound', 'Weimaraner', 'Staffordshire_bullterrier', 'American_Staffordshire_terrier', 'Bedlington_terrier', 'Border_terrier', 'Kerry_blue_terrier', 'Irish_terrier', 'Norfolk_terrier', 'Norwich_terrier', 'Yorkshire_terrier', 'wire-haired_fox_terrier', 'Lakeland_terrier', 'Sealyham_terrier', 'Airedale', 'cairn', 'Australian_terrier', 'Dandie_Dinmont', 'Boston_bull', 'miniature_schnauzer', 'giant_schnauzer', 'standard_schnauzer', 'Scotch_terrier', 'Tibetan_terrier', 'silky_terrier', 'soft-coated_wheaten_terrier', 'West_Highland_white_terrier', 'Lhasa', 'flat-coated_retriever', 'curly-coated_retriever', 'golden_retriever', 'Labrador_retriever', 'Chesapeake_Bay_retriever', 'German_short-haired_pointer', 'vizsla', 'English_setter', 'Irish_setter', 'Gordon_setter', 'Brittany_spaniel', 'clumber', 'English_springer', 'Welsh_springer_spaniel', 'cocker_spaniel', 'Sussex_spaniel', 'Irish_water_spaniel', 'kuvasz', 'schipperke', 'groenendael', 'malinois', 'briard', 'kelpie', 'komondor', 'Old_English_sheepdog', 'Shetland_sheepdog', 'collie', 'Border_collie', 'Bouvier_des_Flandres', 'Rottweiler', 'German_shepherd', 'Doberman', 'miniature_pinscher', 'Greater_Swiss_Mountain_dog', 'Bernese_mountain_dog', 'Appenzeller', 'EntleBucher', 'boxer', 'bull_mastiff', 'Tibetan_mastiff', 'French_bulldog', 'Great_Dane', 'Saint_Bernard', 'Eskimo_dog', 'malamute', 'Siberian_husky', 'affenpinscher', 'basenji', 'pug', 'Leonberg', 'Newfoundland', 'Great_Pyrenees', 'Samoyed', 'Pomeranian', 'chow', 'keeshond', 'Brabancon_griffon', 'Pembroke', 'Cardigan', 'toy_poodle', 'miniature_poodle', 'standard_poodle', 'Mexican_hairless', 'dingo', 'dhole', 'African_hunting_dog'])

print("Classes: ", parms.NUM_CLASSES, 
      "   Labels: ", len(parms.CLASS_NAMES), 
      "  ", parms.CLASS_NAMES)


## Build an input pipeline

In [0]:
import random
import scipy.ndimage
from skimage.filters import gaussian

def image_blur(image):
    # Takes an image and applies Gaussian Blur using skimage filters.
    # Applies random +/- sigma_max to the image

    if bool(np.random.choice([0, 1], p=[0.7, 0.3])):  # change p values as needed . [0., 1.0] is always True
        sigma_max = 3.0
        sigma = random.uniform(0., sigma_max)  # change range or remove if want a fixed sigma value
        image = tf.image.convert_image_dtype(image, dtype=tf.int32)
        image = gaussian(image, sigma=sigma, multichannel=True)
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image

def image_aug_pre_cache(image: tf.Tensor) -> tf.Tensor:

    #######################################################
    # Blur using tf.py_function
    #######################################################
    im_shape = image.shape
    [image,] = tf.py_function(image_blur, [image], [tf.float32])  #parms must be tensors
    image.set_shape(im_shape)
    #######################################################

    image = tf.clip_by_value(image, 0., 1.)  # after majority of augmentations, clip back to 0, 1 before returning
    return image

def image_aug_post_cache(image: tf.Tensor) -> tf.Tensor:

    #######################################################
    # These are native tf.image methods
    #######################################################
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    #######################################################
    # random zoom - random crop + resize which will zoom the image
    #######################################################
    w = parms.IMAGE_COLS
    h = parms.IMAGE_ROWS
    p = 0.90
    image = tf.image.resize(tf.image.random_crop(image, (int(h*p), int(w*p), 3)), (h, w))
    #######################################################

    image = tf.clip_by_value(image, 0., 1.)  # after majority of augmentations, clip back to 0, 1 before returning
    return image

def image_normalize_0_1(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.resize(image, (parms.IMAGE_COLS, parms.IMAGE_ROWS))
    image = tf.cast(image, tf.float32) / 255.
    return image

def image_normalize_0_1_to_1_neg_1(image: tf.Tensor) -> tf.Tensor:
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    return image

def image_normalize_1_to_neg_1(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.resize(image, (parms.IMAGE_ROWS, parms.IMAGE_COLS))
    image = tf.cast(image, tf.float32) / 255.
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    return image

def process_train_pre_cache(image: tf.Tensor, label: tf.Tensor) -> tf.Tensor:
    image = image_normalize_0_1(image)
    image = image_aug_pre_cache(image)
    return image, label_to_onehot(label)

def process_train_post_cache(image: tf.Tensor, label: tf.Tensor) -> tf.Tensor:
    image = image_aug_post_cache(image)
    image = image_normalize_0_1_to_1_neg_1(image) # ImageNet needs 1 to -1
    return image, label

def process_val(image: tf.Tensor, label: tf.Tensor) -> tf.Tensor:
    image = image_normalize_1_to_neg_1(image)
    return image, label_to_onehot(label)

def label_to_onehot(label: tf.Tensor) -> tf.Tensor:
    return tf.one_hot(label, parms.NUM_CLASSES)

In [0]:
# Create Dataset from list of images
train_dataset = datasets['train']
val_dataset = datasets['test']

# split into training and validation sets of images
full_train_len = info.splits['train'].num_examples
full_val_len = info.splits['test'].num_examples

# If doing a sanity check (Small Run), then select a subset of the files
if parms.SMALL_RUN:
    full_train_len_adj = int(full_train_len * 0.2)
    full_val_len_adj = int(full_val_len * 0.05)
else:
    #full_val_len_adj = int(full_val_len * 0.2)
    full_train_len_adj = full_train_len
    full_val_len_adj = full_val_len

images_list_len = full_train_len_adj + full_val_len_adj
train_len = full_train_len_adj
val_len = full_val_len_adj

# Create datasets with new sizes
train_dataset = train_dataset.take(train_len) 
val_dataset = val_dataset.take(val_len)

steps_per_epoch = np.ceil(train_len // parms.BATCH_SIZE) # set step sizes based on train & batch
validation_steps = np.ceil(val_len // parms.BATCH_SIZE) # set step sizes based on val & batch

print("Total number: ", images_list_len, "  Train number: ", train_len, "  Val number: ", val_len)
print("Steps/EPOC: ", steps_per_epoch, "  Steps/Validation: ", validation_steps)

### Training setup

In [0]:
def cache_dataset(dataset, cache=False):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    return dataset

In [0]:
# Verify image paths were loaded and save one path for later in "some_image"
for f in train_dataset.take(2):
    some_image = f[0]
    some_label = f[1]
    print(f[1])

# map training images to pre-cache processing, includes any augmentation
train_dataset = train_dataset.map(process_train_pre_cache, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in train_dataset.take(1):
    print("Map 1 Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())

# cache
train_dataset = cache_dataset(train_dataset, cache="./breedtrain1.tfcache")
#train_dataset = cache_dataset(train_dataset) # no cache

# map training images to post-cache processing, includes any augmentation
train_dataset = train_dataset.map(process_train_post_cache, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in train_dataset.take(1):
    print("Map 2 Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())


# Repeat forever
train_dataset = train_dataset.repeat()

# set the batch size
train_dataset = train_dataset.batch(parms.BATCH_SIZE)


In [0]:
#!rm "./breedval2.tfcache_0.lockfile"
t = np.array([9, 9, 9])
print(t.shape[0])

In [0]:
# Show the images, execute this cell multiple times to see the images

image_batch, label_batch = next(iter(train_dataset))
#show_batch(image_batch.numpy(), label_batch.numpy())
show_batch(image_batch.numpy(), label_batch.numpy(), print_shape=False, number_to_show=6)

### Validation setup

In [0]:
# Verify image paths were loaded and save one path for later in "some_image"
for f in val_dataset.take(2):
    print(f[1])

# map training images to processing, includes any augmentation
val_dataset = val_dataset.map(process_val, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in val_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())

# cache
val_dataset = cache_dataset(val_dataset, cache="./breedval2.tfcache")
#val_dataset = cache_dataset(val_dataset) # no cache

# Repeat forever
val_dataset = val_dataset.repeat()

# set the batch size
val_dataset = val_dataset.batch(parms.BATCH_SIZE)


In [0]:
# Test Validation, use smaller "number_to_show" to help show the augmentation

image_batch, label_batch = next(iter(val_dataset))
#show_batch(image_batch.numpy(), label_batch.numpy())
show_batch(image_batch.numpy(), label_batch.numpy(), number_to_show=6)

## Build  model
- add and validate pretrained model as a baseline

In [0]:
# Create any call backs for training...These are the most common.

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, min_lr=1e-6)
earlystopper = EarlyStopping(patience=8, verbose=1)
checkpointer = ModelCheckpoint(parms.MODEL_PATH, monitor='val_loss', verbose=1, mode="auto", save_best_only=True)
#csv_logger = CSVLogger(self.cvslogfile, append=True, separator=';')

#from keras.callbacks import TensorBoard
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


In [0]:
# Create model and compile it

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D, Conv2DTranspose, Concatenate, Activation
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam, Nadam, SGD
########

# https://www.tensorflow.org/api_docs/python/tf/keras/applications
from tensorflow.keras.applications import MobileNet, imagenet_utils, ResNet50
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

actual_MobileNet = tf.keras.applications.mobilenet.MobileNet()
#actual_ResNet50 = tf.keras.applications.ResNet50()

def set_train_layers(model, train_layers=20): #since 224x224x3, set the first 20 layers of the network to be non-trainable
    if train_layers == 0: #set all non-trainable
        for layer in model.layers:
            layer.trainable=False
    else:
        for layer in model.layers[:train_layers]:             
            layer.trainable=False
        for layer in model.layers[train_layers:]:
            layer.trainable=True
    return model

def predict_image(model_passed, image):  
    image = image_normalize_1_to_neg_1(image)   
    image = np.expand_dims(image, axis=0)
    predictions = model_passed.predict(image)
    return predictions 


def build_model(parms):
    base_model=MobileNet(weights='imagenet',include_top=False, input_shape=parms.IMAGE_DIM) #imports the mobilenet model and discards the last 1000 neuron layer.
    #base_model=ResNet50(weights='imagenet',include_top=False, input_shape=parms.IMAGE_DIM) #imports the ResNet50 model and discards the last 1000 neuron layer.
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dropout(0.3) (x)
    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    #x=Dense(1024,activation='relu')(x) #dense layer 2
    x=Dropout(0.4) (x)
    #x=Dense(512,activation='relu')(x) #dense layer 3
    preds=Dense(parms.NUM_CLASSES, activation=parms.FINAL_ACTIVATION)(x) #final layer
    model=Model(inputs=base_model.input,outputs=preds)
    return model

def compile_model(parms, model):
    # Optimizers: https://ruder.io/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms
    model.compile(loss=parms.LOSS,
          optimizer=SGD(lr=0.001, momentum=0.9),
          #optimizer="adam",
          #optimizer="rmsprop",
          metrics=parms.METRICS)
    return model


In [0]:
# Double check preprocessing with utility.  np.max & np.min should align with your normalization
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py

img_MobileNet = tf.keras.applications.mobilenet.preprocess_input(tf.cast(some_image, tf.float32))
print("Image ", some_image.shape, np.max(some_image), np.min(some_image))
print("Pre   ", img_MobileNet.shape, np.max(img_MobileNet), np.min(img_MobileNet))
fig = plt.figure()
plt.imshow(tf.keras.preprocessing.image.array_to_img(img_MobileNet))
plt.show()

In [0]:
#test an image just using MobileNet
predictions = predict_image(actual_MobileNet, some_image)
result = imagenet_utils.decode_predictions(predictions)

#predictions = predict_image(actual_ResNet50, some_image)
#result = resnet50.decode_predictions(predictions, top=1))

result 


In [0]:
#show the image...
fig = plt.figure()
plt.imshow(tf.keras.preprocessing.image.array_to_img(some_image))
fig.suptitle(parms.CLASS_NAMES[some_label])
plt.show()

## Train model

In [0]:
# Train model
model = build_model(parms)
set_train_layers(model)
model = compile_model(parms, model)

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=parms.EPOCS, 
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[reduce_lr, earlystopper, checkpointer] # include any callbacks...
                    )

In [0]:
# Plot the training history
history_df = pd.DataFrame(history.history)
plt.figure()
history_df[['loss', 'val_loss']].plot(title="Loss")
plt.xlabel('Epocs')
plt.ylabel('Loss')
history_df[['accuracy', 'val_accuracy']].plot(title="Accuracy")
plt.xlabel('Epocs')
plt.ylabel('Accuracy')
plt.show()

## Validate model's predictions
- Create actual_lables and predict_labels
- Calculate Confusion Matrix & Accuracy
- Display results


In [0]:
#Load saved model
from tensorflow.keras.models import load_model 
def load_saved_model(model_path):
    model = load_model(model_path)
    print("loaded: ", model_path)
    return model

model = load_saved_model(parms.MODEL_PATH)

In [0]:
# Use model to generate predicted labels and probabilities
labels, predict_labels, predict_probabilities, bad_results = predictions_using_dataset(model, val_dataset, validation_steps, parms.BATCH_SIZE, create_bad_results_list=False)
#labels, predict_labels, predict_probabilities, bad_results = predictions_using_dataset(model, val_dataset, 1, parms.BATCH_SIZE, create_bad_results_list=False)


In [0]:
show_confusion_matrix(labels, predict_labels, parms.CLASS_NAMES, show_graph=False)

In [0]:
# Graph the results
display_prediction_results(labels, predict_labels, predict_probabilities, parms.NUM_CLASSES, parms.CLASS_NAMES)


In [0]:
#Create a df from the bad results list, can save as csv or use for further analysis
bad_results_df = pd.DataFrame(bad_results, columns =['actual', 'predict', 'prob', 'image'])
bad_results_df.head()

In [0]:
# default is to not return bad_results, change to include them, create_bad_results_list=True

#bad_act, bad_pred, bad_prob, bad_images = zip(*bad_results)


In [0]:
# display images....        
def show_bad_batch(image_batch, bad_act, bad_pred, number_to_show=25):
    plt.figure(figsize=(10,10))
    show_number = number_to_show
    if len(image_batch) < number_to_show:
        show_number = len(image_batch)
        
    for n in range(show_number):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n][0]))
        #s = parms.CLASS_NAMES[bad_pred[n][0]]
        s = "Act: "+ str(bad_act[n][0]) + " Pred: " + str(bad_pred[n][0])
        plt.title(s)
        plt.axis('off')

In [0]:

#show_bad_batch(bad_images, bad_act, bad_pred)